In [12]:
!pip install bs4
import requests
import bs4
import re
from urllib.parse import urljoin

In [13]:
start_link = 'https://ru.wikipedia.org/'
resp = requests.get(start_link)
soup = bs4.BeautifulSoup(resp.content, 'html.parser')
soup.title

<title>Википедия — свободная энциклопедия</title>

Функция для получения всех ссылок с переданной страницы

In [19]:
def extract_all_links(soup, filters, derived_links, start_link, max_width=10):
    page_links = set()
    for link in soup.find_all("a", href=filters):
        link_addr = link.get("href")
        if link_addr:
            # дополнение относительной ссылки до полной:
            link_addr = urljoin(start_link, link_addr)
            # исключение уже имеющихся ссылок, чтобы не получить дерево с "замыканиями":
            if link_addr not in derived_links:
                page_links.add(link_addr)
                if len(page_links) >= max_width:
                    return page_links
    return page_links

Функция для рекурсивного создания дерева ссылок

In [20]:
def extract_tree_of_links(
    page_soup, filters, derived_links, start_link, max_width=10, max_depth=2
):
    tree_of_links = {}
    if max_depth > 0:
        extracted_links = extract_all_links(page_soup, filters, derived_links, start_link, max_width=max_width)
        # множество уже полученных ссылок, чтобы не получить повторов:
        derived_links.update(extracted_links)
        max_depth -= 1
        for link in extracted_links:
            resp = requests.get(link)
            soup = bs4.BeautifulSoup(resp.content, "html.parser")
            links, _ = extract_tree_of_links(
                soup,
                filters,
                derived_links,
                link,
                max_width=max_width,
                max_depth=max_depth,
            )
            tree_of_links[link] = links

    return tree_of_links, derived_links

Получение дерева ссылок с фильтрацией только тех ссылок, которые содержат 4 подряд цифры (перечисление дат текущего дня из стартовой страницы википедии)

In [15]:
# фильтр ссылок с четырехзначным числом, как перечисление дат из раздела "В этот день"
filters = re.compile(r"/\d{4}")
# рекурсивно получаем дерево ссылок с ограничением ширины и глубины
tree, derived_links = extract_tree_of_links(
    soup, filters, set(), start_link, max_width=3, max_depth=3
)

Функция для получения из дерева ссылок конечных листовых элементов, в виде генератора

In [16]:
# функция-генератор "листьев" из дерева ссылок - возвращает по одному "листу" при каждом вызове
def find_leaf(tree):
    for node in tree.items():
        if len(node[1]) == 0:
            yield node[0]
        else:
            yield from find_leaf(node[1])

Обход полученного дерева ссылок, и сохранение в CSV-файл первой попавшейся картинки из листовых элементов дерева, или текста страницы, если там нет картинки

In [18]:
import csv
import base64

# обход дерева ссылок, и получение информации для "листьев"

headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"}
filters = re.compile(r".[jJ][pP][eE]?[gG]")

# создание файла CSV
with open("content.csv", "w", newline="") as csvfile:
    fieldnames = ["link_addr", "type", "base64_content"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames, delimiter="\t")
    writer.writeheader()

for leaf in find_leaf(tree):
    resp = requests.get(leaf)
    soup = bs4.BeautifulSoup(resp.content, "html.parser")
    obj = None
    for img in soup.find_all("img", src=filters):
        # дополнение относительной ссылки до полной:
        img_addr = urljoin(leaf, img.get("src"))
        img_resp = requests.get(img_addr, headers=headers)
        if img_resp.status_code == requests.codes.ok:
            obj = base64.b64encode(img_resp.content)
            type = "img"
            break  # поиск только одного первого изображения

    # если нет ни одной картинки, то сохраним текст страницы
    if obj is None:
        obj = base64.b64encode(soup.text.encode("utf-8"))
        type = "text_utf-8"

    # запись в CSV
    with open("content.csv", "a", newline="") as csvfile:
        fieldnames = ["link_addr", "type", "base64_content"]
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames, delimiter="\t")
        writer.writerow({"link_addr": leaf, "type": type, "base64_content": obj})

В результате получается файл CSV с тремя полями:
* Адрес
* Тип содержимого (img или text_utf-8)
* Содержимое в кодировке Base64

Как разделитель используется символ табуляции


